In [1]:
import datetime as dt
import numpy as np
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [2]:
engine = create_engine("sqlite:///../db/apache.sqlite")
base= automap_base()
base.prepare(engine, reflect=True)
inspector = inspect(engine)
log_table = base.classes.access_response
session = Session(engine)

In [3]:
date_range_2018 = 'select * from access_response where DateTime like "%2018%"'
date_range_2018_results= pd.read_sql_query(date_range_2018, session.bind)

In [4]:
date_range_2018_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,1,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,-,220,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
1,2,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/PMA/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
2,3,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/dbadmin/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
3,4,62.170.2.205,2018-04-08 07:19:26,GMT-0700,HEAD,http://198.27.242.168:80/pma/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
4,5,62.170.2.205,2018-04-08 07:19:26,GMT-0700,HEAD,http://198.27.242.168:80/db/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
5,6,108.41.39.187,2018-04-08 07:53:03,GMT-0700,GET,/phone_pix/cat.jpg,404,-,534,just some guy,40.5833,-73.9240
6,7,74.208.182.37,2018-04-08 09:00:47,GMT-0700,GET,/,200,-,562,Mozilla/4.0 (compatible; MSIE 5.0; Windows NT;...,40.0548,-75.4083
7,8,46.229.164.102,2018-04-08 09:58:37,GMT-0700,GET,/robots.txt,200,-,385,Mozilla/5.0 (compatible; SemrushBot/1.2~bl; +h...,39.0180,-77.5390
8,9,96.57.208.218,2018-04-08 09:59:37,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,-,220,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.7510,-97.8220
9,10,96.57.208.218,2018-04-08 09:59:37,GMT-0700,HEAD,http://198.27.242.168:80/PMA/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.7510,-97.8220


In [5]:
admin_query = 'select * from access_response where URL like "%admin%" and DateTime like "2018%"'
admin_results = pd.read_sql_query(admin_query, session.bind)

In [6]:
admin_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,1,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,-,220,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
1,3,62.170.2.205,2018-04-08 07:19:25,GMT-0700,HEAD,http://198.27.242.168:80/dbadmin/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.1500,15.1833
2,9,96.57.208.218,2018-04-08 09:59:37,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,-,220,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.7510,-97.8220
3,11,96.57.208.218,2018-04-08 09:59:37,GMT-0700,HEAD,http://198.27.242.168:80/dbadmin/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.7510,-97.8220
4,43,46.243.189.99,2018-04-08 22:23:38,GMT-0700,GET,/phpMyAdmin/scripts/setup.php,404,-,478,ZmEu,52.3824,4.8995
5,44,46.243.189.99,2018-04-08 22:23:38,GMT-0700,GET,/phpmyadmin/scripts/setup.php,404,-,478,ZmEu,52.3824,4.8995
6,46,46.243.189.99,2018-04-08 22:23:39,GMT-0700,GET,/myadmin/scripts/setup.php,404,-,477,ZmEu,52.3824,4.8995
7,47,46.243.189.99,2018-04-08 22:23:39,GMT-0700,GET,/MyAdmin/scripts/setup.php,404,-,477,ZmEu,52.3824,4.8995
8,50,107.170.209.132,2018-04-09 00:43:58,GMT-0700,HEAD,http://198.27.242.168:80/phpmyadmin/,404,-,220,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.7312,-122.3826
9,52,107.170.209.132,2018-04-09 00:43:58,GMT-0700,HEAD,http://198.27.242.168:80/dbadmin/,404,-,219,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,37.7312,-122.3826


In [7]:
len(admin_results)

585

In [8]:
setup_query = 'select * from access_response where URL like "%setup%" and DateTime like "2018%"'
setup_results = pd.read_sql_query(setup_query, session.bind)

In [9]:
setup_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,43,46.243.189.99,2018-04-08 22:23:38,GMT-0700,GET,/phpMyAdmin/scripts/setup.php,404,-,478,ZmEu,52.3824,4.8995
1,44,46.243.189.99,2018-04-08 22:23:38,GMT-0700,GET,/phpmyadmin/scripts/setup.php,404,-,478,ZmEu,52.3824,4.8995
2,45,46.243.189.99,2018-04-08 22:23:38,GMT-0700,GET,/pma/scripts/setup.php,404,-,474,ZmEu,52.3824,4.8995
3,46,46.243.189.99,2018-04-08 22:23:39,GMT-0700,GET,/myadmin/scripts/setup.php,404,-,477,ZmEu,52.3824,4.8995
4,47,46.243.189.99,2018-04-08 22:23:39,GMT-0700,GET,/MyAdmin/scripts/setup.php,404,-,477,ZmEu,52.3824,4.8995
5,118,198.167.140.177,2018-04-10 01:10:39,GMT-0700,GET,//phpMyAdmin/scripts/setup.php,404,-,478,-,47.4891,-122.2908
6,119,198.167.140.177,2018-04-10 01:10:39,GMT-0700,GET,//phpmyadmin/scripts/setup.php,404,-,478,-,47.4891,-122.2908
7,120,198.167.140.177,2018-04-10 01:10:39,GMT-0700,GET,//pma/scripts/setup.php,404,-,474,-,47.4891,-122.2908
8,121,198.167.140.177,2018-04-10 01:10:39,GMT-0700,GET,//myadmin/scripts/setup.php,404,-,477,-,47.4891,-122.2908
9,122,198.167.140.177,2018-04-10 01:10:39,GMT-0700,GET,//MyAdmin/scripts/setup.php,404,-,477,-,47.4891,-122.2908


In [10]:
len(setup_results)

477

In [11]:
login_query = 'select * from access_response where URL like "%login%" and DateTime like "2018%"'
login_results = pd.read_sql_query(login_query, session.bind)

In [12]:
login_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,233,113.96.223.207,2018-04-11 20:51:30,GMT-0700,CONNECT,xui.ptlogin2.qq.com:443,405,-,544,-,23.1167,113.2500
1,276,167.99.70.82,2018-04-12 22:03:35,GMT-0700,GET,/login.php,404,-,468,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; ...,1.2931,103.8558
2,309,91.200.12.56,2018-04-13 20:26:26,GMT-0700,GET,/wp-login.php,404,-,529,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,50.4500,30.5233
3,310,91.200.12.56,2018-04-13 20:26:26,GMT-0700,GET,//wp-login.php,404,-,472,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,50.4500,30.5233
4,312,167.99.70.82,2018-04-13 21:39:06,GMT-0700,GET,/login.php,404,-,468,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; ...,1.2931,103.8558
5,556,212.21.66.6,2018-04-03 03:59:40,GMT-0700,GET,/wp-login.php?action=register,404,-,448,Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/2...,52.1638,11.6333
6,571,185.234.218.250,2018-04-03 13:57:31,GMT-0700,GET,/wp-login.php,404,http://spinster.org/wp-login.php,472,Mozilla/4.0 (compatible; MSIE 9.0; Windows NT ...,53.3472,-6.2439
7,572,185.234.218.250,2018-04-03 13:57:32,GMT-0700,GET,/wp-login.php,404,http://spinster.org/wp-login.php,472,Mozilla/4.0 (compatible; MSIE 9.0; Windows NT ...,53.3472,-6.2439
8,1301,91.200.12.65,2018-03-28 01:10:56,GMT-0700,GET,/wp-login.php,404,-,529,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,50.4500,30.5233
9,1302,91.200.12.65,2018-03-28 01:10:56,GMT-0700,GET,//wp-login.php,404,-,472,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,50.4500,30.5233


In [13]:
len(login_results)

57

In [14]:
test_query = 'select * from access_response where URL like "%test%" and DateTime like "2018%"'
test_results = pd.read_sql_query(test_query, session.bind)

In [15]:
test_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,1925,188.36.104.226,2018-03-19 02:16:17,GMT-0700,GET,/phpTest/zologize/axa.php,404,-,505,-,47.5223,19.0564
1,2896,89.19.24.154,2018-03-16 12:30:59,GMT-0700,GET,/cgi-bin/test.cgi,404,-,478,-,41.0214,28.9948
2,2923,43.240.66.47,2018-03-16 17:56:51,GMT-0700,GET,/cgi-bin/test.cgi,404,-,478,-,20.0000,77.0000
3,2962,85.118.206.11,2018-03-17 04:36:51,GMT-0700,GET,/cgi-bin/test.cgi,404,-,478,-,59.3247,18.0560
4,3527,89.19.24.154,2018-03-10 10:36:55,GMT-0800,GET,/cgi-bin/test.cgi,404,-,478,-,41.0214,28.9948
5,3613,222.186.24.113,2018-02-25 19:42:41,GMT-0800,GET,/cgi-bin/test.cgi,404,-,478,-,32.0617,118.7778
6,3622,84.244.180.102,2018-02-25 19:47:37,GMT-0800,GET,/cgi-bin/test.cgi,404,-,478,-,52.3824,4.8995
7,3769,195.22.126.117,2018-02-28 21:36:13,GMT-0800,GET,/cgi-bin/test-cgi,404,-,535,the beast,52.2394,21.0362
8,5074,84.244.180.102,2018-02-21 18:53:37,GMT-0800,GET,/cgi-bin/test.cgi,404,-,478,-,52.3824,4.8995
9,5082,212.34.138.31,2018-02-21 19:16:06,GMT-0800,GET,/cgi-bin/test.cgi,404,-,478,-,40.4172,-3.6840


In [16]:
len(test_results)

13

In [17]:
login_query = 'select * from access_response where URL like "%login%" and DateTime like "2018%"'
login_results = pd.read_sql_query(login_query, session.bind)

In [18]:
login_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,233,113.96.223.207,2018-04-11 20:51:30,GMT-0700,CONNECT,xui.ptlogin2.qq.com:443,405,-,544,-,23.1167,113.2500
1,276,167.99.70.82,2018-04-12 22:03:35,GMT-0700,GET,/login.php,404,-,468,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; ...,1.2931,103.8558
2,309,91.200.12.56,2018-04-13 20:26:26,GMT-0700,GET,/wp-login.php,404,-,529,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,50.4500,30.5233
3,310,91.200.12.56,2018-04-13 20:26:26,GMT-0700,GET,//wp-login.php,404,-,472,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,50.4500,30.5233
4,312,167.99.70.82,2018-04-13 21:39:06,GMT-0700,GET,/login.php,404,-,468,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; ...,1.2931,103.8558
5,556,212.21.66.6,2018-04-03 03:59:40,GMT-0700,GET,/wp-login.php?action=register,404,-,448,Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/2...,52.1638,11.6333
6,571,185.234.218.250,2018-04-03 13:57:31,GMT-0700,GET,/wp-login.php,404,http://spinster.org/wp-login.php,472,Mozilla/4.0 (compatible; MSIE 9.0; Windows NT ...,53.3472,-6.2439
7,572,185.234.218.250,2018-04-03 13:57:32,GMT-0700,GET,/wp-login.php,404,http://spinster.org/wp-login.php,472,Mozilla/4.0 (compatible; MSIE 9.0; Windows NT ...,53.3472,-6.2439
8,1301,91.200.12.65,2018-03-28 01:10:56,GMT-0700,GET,/wp-login.php,404,-,529,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,50.4500,30.5233
9,1302,91.200.12.65,2018-03-28 01:10:56,GMT-0700,GET,//wp-login.php,404,-,472,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,50.4500,30.5233


In [19]:
ramble_query = 'select * from access_response where URL like "%ramble%" and DateTime like "2018%"'
ramble_results = pd.read_sql_query(ramble_query, session.bind)

In [20]:
ramble_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,228,216.244.66.229,2018-04-11 16:26:38,GMT-0700,GET,/ramble/index.php?blogId=16,404,-,476,Mozilla/5.0 (compatible; DotBot/1.1; http://ww...,47.4891,-122.2908
1,280,217.69.133.4,2018-04-13 01:28:51,GMT-0700,GET,/ramble/index.php?blogId=23,404,-,469,Mozilla/5.0 (compatible; Linux x86_64; Mail.RU...,55.7386,37.6068
2,281,217.69.133.2,2018-04-13 01:31:02,GMT-0700,GET,/ramble/index.php?blogId=5,404,-,469,Mozilla/5.0 (compatible; Linux x86_64; Mail.RU...,55.7386,37.6068
3,526,216.244.66.229,2018-04-02 08:42:22,GMT-0700,GET,/ramble/index.php?blogId=50,404,-,476,Mozilla/5.0 (compatible; DotBot/1.1; http://ww...,47.4891,-122.2908
4,538,216.244.66.246,2018-04-02 14:25:14,GMT-0700,GET,/ramble/index.php?blogId=101,404,-,476,Mozilla/5.0 (compatible; DotBot/1.1; http://ww...,47.4891,-122.2908
5,539,216.244.66.246,2018-04-02 14:27:15,GMT-0700,GET,/ramble/index.php?blogId=82,404,-,476,Mozilla/5.0 (compatible; DotBot/1.1; http://ww...,47.4891,-122.2908
6,544,185.234.218.86,2018-04-02 19:02:30,GMT-0700,GET,/ramble/index.php?blogId=54,404,-,469,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,53.3472,-6.2439
7,714,92.100.184.21,2018-04-05 05:20:16,GMT-0700,GET,/ramble/index.php?blogid=57,404,-,507,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5...,59.8944,30.2642
8,718,216.244.66.246,2018-04-05 07:02:55,GMT-0700,GET,/ramble/index.php?blogId=82,404,-,476,Mozilla/5.0 (compatible; DotBot/1.1; http://ww...,47.4891,-122.2908
9,721,216.244.66.246,2018-04-05 08:22:20,GMT-0700,GET,/ramble/index.php?blogId=101,404,-,476,Mozilla/5.0 (compatible; DotBot/1.1; http://ww...,47.4891,-122.2908


In [21]:
len(ramble_results)

158

In [22]:
port_query = 'select * from access_response where URL like "%port%" and DateTime like "2018%"'
port_results = pd.read_sql_query(port_query, session.bind)

In [23]:
port_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,18,177.105.65.146,2018-04-08 10:36:47,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,-8.0500,-34.9000
1,25,61.155.61.149,2018-04-08 13:05:00,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,32.0617,118.7778
2,30,222.74.34.5,2018-04-08 15:01:37,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,40.6522,109.8222
3,60,113.161.131.53,2018-04-09 05:06:24,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,10.3500,107.0667
4,73,125.93.51.226,2018-04-09 15:54:31,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,23.1167,113.2500
5,155,115.159.151.108,2018-04-10 18:40:16,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,39.9289,116.3883
6,248,210.209.87.83,2018-04-12 03:07:01,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,22.2500,114.1667
7,257,190.46.9.59,2018-04-12 11:13:59,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,-33.0081,-71.5197
8,286,139.129.221.52,2018-04-13 04:07:12,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,30.2936,120.1614
9,307,217.115.86.6,2018-04-13 19:49:19,GMT-0700,POST,/wls-wsat/CoordinatorPortType,404,-,509,Mozilla/5.0 (Windows NT 6.1; rv:5.0) Gecko/201...,55.1172,61.6282


In [24]:
len(port_results)

248

In [25]:
ua_script_query = 'select * from access_response where UserAgent like "%urll%" or UserAgent like "%curl%" and DateTime like "2018%"'
ua_script_results = pd.read_sql_query(ua_script_query, session.bind)

In [26]:
len(ua_script_results)

330

In [27]:
ua_script_results

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
0,26,24.48.238.249,2018-04-08 13:10:40,GMT-0700,GET,/phone_pix/cat.jpg,404,-,496,Python-urllib/3.6,18.2115,-67.1026
1,27,24.48.238.249,2018-04-08 13:35:08,GMT-0700,GET,/phone_pix/cat.jpg,404,-,496,Python-urllib/3.6,18.2115,-67.1026
2,32,185.207.205.93,2018-04-08 18:53:52,GMT-0700,HEAD,/,200,-,260,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
3,33,185.207.205.93,2018-04-08 18:53:53,GMT-0700,GET,/,200,-,543,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
4,55,114.147.72.16,2018-04-09 01:11:06,GMT-0700,GET,/phone_pix/cat.jpg,404,-,496,Python-urllib/3.6,34.6864,135.5200
5,92,185.207.205.93,2018-04-09 19:57:51,GMT-0700,HEAD,/,200,-,260,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
6,93,185.207.205.93,2018-04-09 19:57:52,GMT-0700,GET,/,200,-,543,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
7,136,135.245.115.103,2018-04-10 13:03:14,GMT-0700,GET,/phone_pix/cat.jpg,404,-,477,Python-urllib/1.17,41.8032,-88.1341
8,151,12.156.67.130,2018-04-10 15:44:47,GMT-0700,GET,/phone_pix/cat.jpg,404,-,477,curl/7.47.0,37.7510,-97.8220
9,159,185.207.205.93,2018-04-10 20:59:59,GMT-0700,HEAD,/,200,-,260,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995


In [28]:
ua_script_results_no_cat = ua_script_results[(ua_script_results.URL != '/phone_pix/cat.jpg')]

In [29]:
ua_script_results_no_cat

,id,ip,DateTime,TimeZone,Method,URL,ResponseCode,Referer,BytesSent,UserAgent,latitude,longitude
2,32,185.207.205.93,2018-04-08 18:53:52,GMT-0700,HEAD,/,200,-,260,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
3,33,185.207.205.93,2018-04-08 18:53:53,GMT-0700,GET,/,200,-,543,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
5,92,185.207.205.93,2018-04-09 19:57:51,GMT-0700,HEAD,/,200,-,260,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
6,93,185.207.205.93,2018-04-09 19:57:52,GMT-0700,GET,/,200,-,543,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
9,159,185.207.205.93,2018-04-10 20:59:59,GMT-0700,HEAD,/,200,-,260,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
10,160,185.207.205.93,2018-04-10 20:59:59,GMT-0700,GET,/,200,-,543,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
13,239,185.207.205.93,2018-04-11 21:45:29,GMT-0700,HEAD,/,200,-,260,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
14,240,185.207.205.93,2018-04-11 21:45:29,GMT-0700,GET,/,200,-,543,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
16,277,185.207.205.93,2018-04-12 22:37:37,GMT-0700,HEAD,/,200,-,260,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995
17,278,185.207.205.93,2018-04-12 22:37:37,GMT-0700,GET,/,200,-,543,PycURL/7.19.5 libcurl/7.38.0 GnuTLS/3.3.8 zlib...,52.3824,4.8995


In [30]:
len(ua_script_results_no_cat)

246